### ModernBERT Reference  
Warner et al., “Smarter, Better, Faster, Longer: A Modern Bidirectional Encoder…”, arXiv:2412.13663 (2024).

@misc{modernbert,
  title={Smarter, Better, Faster, Longer: A Modern Bidirectional Encoder for Fast, Memory Efficient, and Long Context Finetuning and Inference},
  author={Benjamin Warner and Antoine Chaffin and Benjamin Clavié and Orion Weller and Oskar Hallström and Said Taghadouini and Alexis Gallagher and Raja Biswas and Faisal Ladhak and Tom Aarsen and Nathan Cooper and Griffin Adams and Jeremy Howard and Iacopo Poli},
  year={2024},
  eprint={2412.13663},
  archivePrefix={arXiv},
  primaryClass={cs.CL},
  url={https://arxiv.org/abs/2412.13663 }
}

In [1]:
# imports
import os, datetime, warnings, re, unicodedata, pandas as pd
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm
import random, numpy as np, torch
random.seed(42); np.random.seed(42); torch.manual_seed(42)

from torch import nn, optim
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModel,
    get_linear_schedule_with_warmup,
    DataCollatorWithPadding
)
from torch.utils.tensorboard import SummaryWriter  

logdir = f'tb_logs/run_{datetime.datetime.now():%Y%m%d-%H%M%S}'
os.makedirs(logdir, exist_ok=True)
writer = SummaryWriter(logdir)
print(f'TensorBoard logs → {logdir}')

os.environ['TOKENIZERS_PARALLELISM'] = 'false'
warnings.filterwarnings('ignore', message=r'Some known HF warning regex')

df = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/train.csv')
pd.set_option('display.max_colwidth', None)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Running on:', device)

2025-08-17 20:13:33.301857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755461613.678793      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755461613.783263      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorBoard logs → tb_logs/run_20250817-201349
Running on: cuda


In [2]:
# 0. helper to normalise whitespace/Unicode
def _clean(txt: str) -> str:
    txt = unicodedata.normalize('NFKC', txt)
    txt = re.sub(r'\s+', ' ', txt)
    return txt.strip()

# 1. minimal preprocessing
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # text fields
    df['StudentExplanation'] = (
        df['StudentExplanation'].fillna('').apply(_clean)
    )
    df['QuestionText'] = df['QuestionText'].apply(_clean)
    df['MC_Answer']    = df['MC_Answer'].apply(_clean)

    # misconception field
    df['Misconception'] = (
        df['Misconception']
          .fillna('NA')
          .astype(str)
          .str.strip()
          .replace({'Wrong_fraction': 'Wrong_Fraction'})
    )
    mask = df['Category'].str.endswith('Misconception')
    df.loc[~mask, 'Misconception'] = 'NA'

    # joint label string
    df['label_str'] = df['Category'] + ':' + df['Misconception']
    return df

# 2. build label maps + attach label_id
def build_label_maps(df: pd.DataFrame):
    labels = sorted(df['label_str'].unique())
    label2id = {lbl: i for i, lbl in enumerate(labels)}
    id2label = {i: lbl for lbl, i in label2id.items()}
    df['label_id'] = df['label_str'].map(label2id).astype(int)
    return df, label2id, id2label

# 3. run the pipeline
df = preprocess(df)                     
df, label2id, id2label = build_label_maps(df)   

# 4. stratified K-fold
warnings.filterwarnings('ignore', message='The least populated class')

k = 5
skf = StratifiedKFold(
    n_splits=k,
    shuffle=True,
    random_state=42
)

df['fold'] = -1
for fold, (_, val_idx) in enumerate(skf.split(df, y=df['label_id'])):
    df.loc[val_idx, 'fold'] = fold

# sanity check
assert (df['fold'] >= 0).all()
print(df['fold'].value_counts().sort_index())

fold
0    7340
1    7339
2    7339
3    7339
4    7339
Name: count, dtype: int64


In [3]:
# 5. loading ModernBERT tokenizer
MODEL_NAME = 'answerdotai/ModernBERT-base'
NUM_LABELS = len(label2id)
MAX_LEN = 256

# 6. fast tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True
)
SPECIAL_TOKENS = ['<Q>', '</Q>', '<A>', '</A>', '<E>', '</E>']
tokenizer.add_special_tokens({'additional_special_tokens': SPECIAL_TOKENS})
for tok in SPECIAL_TOKENS:
    assert tokenizer.convert_tokens_to_ids(tok) != tokenizer.unk_token_id, f'{tok} is UNK!'
    
# 7. template builder
TEMPLATE = '<Q> {q} </Q> <A> {a} </A> <E> {e} </E>'

def build_text(row):
    return TEMPLATE.format(
        q=row['QuestionText'],
        a=row['MC_Answer'],
        e=row['StudentExplanation']
    )

df['text'] = df.apply(build_text, axis=1)

# 8. sampling sequence lenght distribution
tok_lens = df['text'].apply(lambda s: len(tokenizer.tokenize(s)))
print(tok_lens.describe(percentiles=[.5,.75,.9,.95,.99]))

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

count    36696.000000
mean        63.591699
std         18.033133
min         26.000000
50%         62.000000
75%         74.000000
90%         87.000000
95%         98.000000
99%        114.000000
max        222.000000
Name: text, dtype: float64


In [4]:
# 9. PyTorch Dataset  ─ no token_type_ids for ModernBERT
class MAPDataset(torch.utils.data.Dataset):
    def __init__(self, frame, tokenizer):
        self.encodings = tokenizer(
            frame['text'].tolist(),
            truncation=True,
            max_length=MAX_LEN,
            return_attention_mask=True,
            return_token_type_ids=False,
        )
        self.labels = frame['label_id'].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}  
        item['labels'] = self.labels[idx]                      
        return item

# 10. dataframe → DataLoader
collate_fn = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

def make_loader(frame, batch_size=16, shuffle=True, num_workers=2):
    ds = MAPDataset(frame, tokenizer)

    kwargs = dict(
        dataset=ds,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=collate_fn,
        pin_memory=True,
        num_workers=num_workers,
    )
    if num_workers > 0:
        kwargs.update(dict(persistent_workers=True, prefetch_factor=2))

    return DataLoader(**kwargs)


'''
# 11. hold-out fold
val_fold = 4

train_df = df[df['fold'] != val_fold].reset_index(drop=True)
val_df = df[df['fold'] == val_fold].reset_index(drop=True)
print(len(train_df), 'train rows |', len(val_df), 'val rows')

train_loader = make_loader(train_df, batch_size=16, shuffle=True,  num_workers=2)
val_loader = make_loader(val_df, batch_size=32, shuffle=False, num_workers=2)
'''

"\n# 11. hold-out fold\nval_fold = 4\n\ntrain_df = df[df['fold'] != val_fold].reset_index(drop=True)\nval_df = df[df['fold'] == val_fold].reset_index(drop=True)\nprint(len(train_df), 'train rows |', len(val_df), 'val rows')\n\ntrain_loader = make_loader(train_df, batch_size=16, shuffle=True,  num_workers=2)\nval_loader = make_loader(val_df, batch_size=32, shuffle=False, num_workers=2)\n"

In [5]:
MODEL_NAME = 'answerdotai/ModernBERT-base'
NUM_LABELS = len(label2id)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 12. ModernBERT + single-layer classification head (no token_type_ids)
class ModernBertClassifier(nn.Module):
    def __init__(self, num_labels: int = NUM_LABELS, dropout: float = 0.1):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(MODEL_NAME)
        self.encoder.resize_token_embeddings(len(tokenizer))
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.encoder.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_vec = out.last_hidden_state[:, 0]                     
        logits = self.classifier(self.dropout(cls_vec))

        loss = None
        if labels is not None:
            loss = nn.functional.cross_entropy(logits, labels)

        return {'logits': logits, 'loss': loss}

# 13. factory → model, optimizer, scheduler
def build_model(total_train_steps, lr=2e-5, weight_decay=0.01, warmup_ratio=0.1):
    model = ModernBertClassifier().to(device)

    # weight-decay only on non-bias / non-LayerNorm parameters
    no_decay = {'bias', 'LayerNorm.weight'}
    param_groups = [
        {
            'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': weight_decay,
        },
        {
            'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0,
        },
    ]

    optimizer = optim.AdamW(param_groups, lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(total_train_steps * warmup_ratio),
        num_training_steps=total_train_steps,
    )
    return model, optimizer, scheduler

In [ ]:
# 14. training the model + val metrics MAP@3 — AMP version
'''
%load_ext tensorboard
%tensorboard --logdir tb_logs --host 0.0.0.0

epochs = 10
total_steps = len(train_loader) * epochs
model, optim, sched = build_model(total_train_steps=total_steps)
scaler = torch.cuda.amp.GradScaler()                     

for epoch in range(epochs):
    model.train()
    running = 0.0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False):
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.cuda.amp.autocast():                  
            out  = model(**batch)
            loss = out['loss']

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  
        scaler.step(optim)
        scaler.update()
        sched.step()
        optim.zero_grad(set_to_none=True)

        running += loss.item()

    avg_train = running / len(train_loader)
    writer.add_scalar('Loss/train', avg_train, epoch)

    # validation
    model.eval()
    val_running, preds, y_true = 0.0, [], []
    with torch.no_grad(), torch.cuda.amp.autocast():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**batch)

            val_running += out['loss'].item()
            preds.append(out['logits'].float().cpu())    
            y_true.extend(batch['labels'].cpu())

    avg_val = val_running / len(val_loader)
    writer.add_scalar('Loss/val', avg_val, epoch)

    # MAP@3
    preds = torch.vstack(preds)                          
    top3 = torch.topk(preds, 3, dim=1).indices.cpu()    
    hits = (top3 == torch.tensor(y_true).unsqueeze(1))  
    ranks = hits.float() * (1 / (torch.arange(1, 4).float()))  
    map3 = ranks.sum(dim=1).mean().item()

    writer.add_scalar('MAP3/val', map3, epoch)
    print(f'Epoch {epoch+1}: train_loss={avg_train:.4f}  '
          f'val_loss={avg_val:.4f}  MAP@3={map3:.4f}')
'''

# --- helpers ---
def predict_logits(model, loader):
    model.eval()
    outs = []
    with torch.no_grad(), torch.amp.autocast('cuda'):
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outs.append(model(**batch)["logits"].float().cpu())
    return torch.vstack(outs)  # (N, NUM_LABELS)

def map3_from_logits(logits, y_true):
    top3 = torch.topk(logits, 3, dim=1).indices.cpu()        # (N, 3)
    y_true = torch.as_tensor(y_true).unsqueeze(1)            # (N, 1)
    hits = (top3 == y_true)                                  # boolean
    ranks = hits.float() * (1.0 / torch.arange(1, 4).float())# [1, 1/2, 1/3]
    return ranks.sum(dim=1).mean().item()

# --- 5-fold training + OOF collection with best-on-val checkpointing ---
epochs = 10
patience = 2  # early stopping after 'patience' epochs without MAP@3 improvement
fold_scores = []
oof_logits = torch.zeros((len(df), NUM_LABELS), dtype=torch.float32)
checkpoints = []

for val_fold in range(5):
    # precise split that preserves original indices for OOF placement
    val_idx = df.index[df["fold"] == val_fold].to_numpy()
    val_df  = df.loc[val_idx].reset_index(drop=True)
    train_df = df[df["fold"] != val_fold].reset_index(drop=True)

    train_loader = make_loader(train_df, batch_size=16, shuffle=True)
    val_loader = make_loader(val_df, batch_size=32, shuffle=False)

    # fresh model/opt/sched/scaler
    total_steps = len(train_loader) * epochs
    model, optimizer, sched = build_model(total_train_steps=total_steps)
    scaler = torch.amp.GradScaler('cuda')

    best_map3 = -1.0
    best_val_logits = None
    best_path = f"/kaggle/working/modernbert_fold{val_fold}_best.pt"
    no_improve = 0

    # ----- train -----
    for epoch in range(epochs):
        model.train()
        running = 0.0

        for batch in tqdm(train_loader, desc=f"F{val_fold} E{epoch+1}/{epochs}", leave=False):
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.amp.autocast('cuda'):
                out = model(**batch)
                loss = out["loss"]

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            sched.step()
            optimizer.zero_grad(set_to_none=True)

            running += loss.item()

        # ----- end-of-epoch validation -----
        val_logits = predict_logits(model, val_loader)
        y_true = val_df["label_id"].tolist()
        cur_map3 = map3_from_logits(val_logits, y_true)
        print(f"Fold {val_fold} Epoch {epoch+1}: train_loss={running/len(train_loader):.4f}  MAP@3={cur_map3:.4f}")

        # checkpoint the best
        if cur_map3 > best_map3:
            best_map3 = cur_map3
            best_val_logits = val_logits
            torch.save(model.state_dict(), best_path)
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping on fold {val_fold} at epoch {epoch+1}")
                break

    # OOF using the best epoch's logits, aligned to original df indices (all Torch)
    val_idx_t = torch.as_tensor(val_idx, dtype=torch.long)
    if best_val_logits is None:
        best_val_logits = predict_logits(model, val_loader)
        y_true = val_df["label_id"].tolist()
        best_map3 = map3_from_logits(best_val_logits, y_true)
        torch.save(model.state_dict(), best_path)  
        
    oof_logits.index_copy_(0, val_idx_t, best_val_logits)        
    fold_scores.append(best_map3)
    checkpoints.append(best_path)
    
    # ---- free some RAM/VRAM before next fold ----
    del train_loader, val_loader        
    del model, optimizer, sched         
    import gc; gc.collect()            
    torch.cuda.empty_cache()            

print("CV MAP@3 (mean over folds):", sum(fold_scores)/len(fold_scores))
print("Best checkpoints:", checkpoints)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


F0 E1/10:   0%|          | 0/1835 [00:00<?, ?it/s]

Fold 0 Epoch 1: train_loss=1.3544  MAP@3=0.8297


...

F0 E3/10:   0%|          | 0/1835 [00:00<?, ?it/s]

Fold 0 Epoch 3: train_loss=0.5030  MAP@3=0.9047


F0 E4/10:   0%|          | 0/1835 [00:00<?, ?it/s]

Fold 0 Epoch 4: train_loss=0.4150  MAP@3=0.9139


F0 E5/10:   0%|          | 0/1835 [00:00<?, ?it/s]